# [Housing Prices Competition for Kaggle Learn Users](https://www.kaggle.com/c/home-data-for-ml-course)

In [1]:
# Code you have previously used to load data
import pandas as pd
import numpy as np

from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

from sklearn import linear_model

from sklearn.metrics import mean_absolute_error, r2_score

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

In [2]:
home_data = pd.read_csv("./data/hpc_train.csv")
home_data

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [3]:
home_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [4]:
y = home_data.SalePrice
y

0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1460, dtype: int64

## Linear Regression

In [5]:
def save_submission_linear(model_name, model, features):
    # read test data file using pandas
    test_data = pd.read_csv("./data/hpc_test.csv")

    # create test_X which comes from test_data but includes only the columns you used for prediction.
    # The list of columns is stored in a variable called features

    test_X = test_data[features]

    # make predictions which we will submit. 
    test_preds = model.predict(test_X)

    # The lines below shows you how to save your data in the format needed to score it in the competition
    output = pd.DataFrame({'Id': test_data.Id,
                           'SalePrice': test_preds})
    
    output_file = f'submission_{model_name}.csv'
    output.to_csv(output_file, index=False)
    
    print(f'{output_file} saved!')

In [6]:
home_data_num = home_data.select_dtypes(include=[np.number])
home_data_num.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 38 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   LotFrontage    1201 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   OverallQual    1460 non-null   int64  
 5   OverallCond    1460 non-null   int64  
 6   YearBuilt      1460 non-null   int64  
 7   YearRemodAdd   1460 non-null   int64  
 8   MasVnrArea     1452 non-null   float64
 9   BsmtFinSF1     1460 non-null   int64  
 10  BsmtFinSF2     1460 non-null   int64  
 11  BsmtUnfSF      1460 non-null   int64  
 12  TotalBsmtSF    1460 non-null   int64  
 13  1stFlrSF       1460 non-null   int64  
 14  2ndFlrSF       1460 non-null   int64  
 15  LowQualFinSF   1460 non-null   int64  
 16  GrLivArea      1460 non-null   int64  
 17  BsmtFullBath   1460 non-null   int64  
 18  BsmtHalf

In [7]:
def use_modelession_and_save_submission_linear(model_type, features):
    X = home_data_num[features]
    print('X:', X)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=100)
    model = linear_model.LinearRegression()
    model.fit(X_train, y_train)
    print('Coefficients:', model.coef_)
    print('Intercept:', model.intercept_)

    y_predicted = model.predict(X_test)

    print(f'R2: {r2_score(y_true=y_test, y_pred=y_predicted)}')
    print(f"MSE: {mean_absolute_error(y_true=y_test, y_pred=y_predicted)}")
    
    save_submission_linear(f'{model_type}_modelession', model, features)

### Simple Linear Regression

In [8]:
features = ['OverallQual']

use_modelession_and_save_submission_linear(model_type='simple', features=features)

X:       OverallQual
0               7
1               6
2               7
3               7
4               8
...           ...
1455            6
1456            6
1457            7
1458            5
1459            5

[1460 rows x 1 columns]
Coefficients: [44780.7500158]
Intercept: -92550.08368564205
R2: 0.647414965422141
MSE: 32228.631737310807
submission_simple_modelession.csv saved!


### Multiple Linear Regression

In [9]:
features = ['OverallQual', 
            'GrLivArea', 
            '1stFlrSF',
            'YearBuilt',
            'FullBath',
            'Fireplaces'
#             'GarageCars', 
#             'TotalBsmtSF',
#             'GarageArea'
           ]

use_modelession_and_save_submission_linear(model_type='multiple', features=features)

X:       OverallQual  GrLivArea  1stFlrSF  YearBuilt  FullBath  Fireplaces
0               7       1710       856       2003         2           0
1               6       1262      1262       1976         2           1
2               7       1786       920       2001         2           1
3               7       1717       961       1915         1           1
4               8       2198      1145       2000         2           1
...           ...        ...       ...        ...       ...         ...
1455            6       1647       953       1999         2           1
1456            6       2073      2073       1978         2           2
1457            7       2340      1188       1941         2           2
1458            5       1078      1078       1950         1           0
1459            5       1256      1256       1965         1           0

[1460 rows x 6 columns]
Coefficients: [22835.19807868    47.65242266    30.13923009   506.74100878
 -4903.09018692 10232.02274835]
I

## Multiple Polynomial Regression

In [10]:
def save_submission_linear(model_name, model, features, poly):
    # read test data file using pandas
    test_data = pd.read_csv("./data/hpc_test.csv")

    # create test_X which comes from test_data but includes only the columns you used for prediction.
    # The list of columns is stored in a variable called features

    test_X = test_data[features]
    
    X_test_poly = poly.fit_transform(test_X)

    # make predictions which we will submit. 
    test_preds = model.predict(X_test_poly)

    # The lines below shows you how to save your data in the format needed to score it in the competition
    output = pd.DataFrame({'Id': test_data.Id,
                           'SalePrice': test_preds})
    
    output_file = f'submission_{model_name}.csv'
    output.to_csv(output_file, index=False)
    
    print(f'{output_file} saved!')

In [11]:
def use_polynomial_regression_and_save_submission_linear(features):
    X = home_data_num[features]
    print('X:', X)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=100)
    
    
    for degree in range(2, 10):
        poly = PolynomialFeatures(degree=degree)
        X_train_poly = poly.fit_transform(X_train)

        model = linear_model.LinearRegression()
        model.fit(X_train_poly, y_train)
#         print('Coefficients:', model.coef_)
#         print('Intercept:', model.intercept_)
        
        X_test_poly = poly.fit_transform(X_test)
        y_predicted = model.predict(X_test_poly)

        print(f'R2: {r2_score(y_true=y_test, y_pred=y_predicted)}')
        print(f"MSE: {mean_absolute_error(y_true=y_test, y_pred=y_predicted)}")

        save_submission_linear(f'polynomial-{degree}_regression', 
                               model, 
                               features, 
                               poly)
        
        print()

In [12]:
from sklearn.preprocessing import PolynomialFeatures

features = ['OverallQual', 
            'GrLivArea', 
            '1stFlrSF',
            'YearBuilt',
            'FullBath',
            'Fireplaces'
           ]

use_polynomial_regression_and_save_submission_linear(features=features)

X:       OverallQual  GrLivArea  1stFlrSF  YearBuilt  FullBath  Fireplaces
0               7       1710       856       2003         2           0
1               6       1262      1262       1976         2           1
2               7       1786       920       2001         2           1
3               7       1717       961       1915         1           1
4               8       2198      1145       2000         2           1
...           ...        ...       ...        ...       ...         ...
1455            6       1647       953       1999         2           1
1456            6       2073      2073       1978         2           2
1457            7       2340      1188       1941         2           2
1458            5       1078      1078       1950         1           0
1459            5       1256      1256       1965         1           0

[1460 rows x 6 columns]
R2: 0.8332637628481606
MSE: 22260.93759671832
submission_polynomial-2_regression.csv saved!

R2: 0.780438600

## XGBoost

In [13]:
# features = ['LotArea', 
#             'YearBuilt', 
#             '1stFlrSF', 
#             '2ndFlrSF', 
#             'FullBath', 
#             'BedroomAbvGr', 
#             'TotRmsAbvGrd']

features = ['OverallQual', 
            'GrLivArea', 
            '1stFlrSF',
            'YearBuilt',
            'FullBath',
            'Fireplaces'
           ]

X = home_data[features]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [29]:
#XGBoost hyper-parameter tuning
def hyperParameterTuning(X_train, y_train):
    param_tuning = {
        'learning_rate': [0.01, 0.1],
        'max_depth': [x for x in range(2, 11, 1)],
        'min_child_weight': [1, 3, 5],
        'subsample': [0.5, 0.7],
        'colsample_bytree': [0.5, 0.7],
        'n_estimators' : [x for x in range(100, 1000, 100)],
        'objective': ['reg:squarederror']
    }
    
    xgb_model = XGBRegressor()

    gsearch = GridSearchCV(estimator = xgb_model,
                           param_grid = param_tuning,                        
                           scoring = 'neg_mean_absolute_error', #MAE
                           #scoring = 'neg_mean_squared_error',  #MSE
                           cv = 5,
                           n_jobs = -1,
                           verbose = 1)

    gsearch.fit(X_train, y_train)

    return gsearch.best_params_

In [30]:
# Run only in the first run of the kernel
hyperParameterTuning(X_train, y_train)

Fitting 5 folds for each of 1944 candidates, totalling 9720 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  52 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 352 tasks      | elapsed:   15.6s
[Parallel(n_jobs=-1)]: Done 852 tasks      | elapsed:   40.8s
[Parallel(n_jobs=-1)]: Done 1500 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 1950 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 2500 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 3760 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 4568 tasks      | elapsed:  5.4min
[Parallel(n_jobs=-1)]: Done 5872 tasks      | elapsed:  6.8min
[Parallel(n_jobs=-1)]: Done 7040 tasks      | elapsed:  8.5min
[Parallel(n_jobs=-1)]: Done 8758 tasks      | elapsed: 10.5min
[Parallel(n_jobs=-1)]: Done 9720 out of 9720 | elapsed: 12.2min finished


{'colsample_bytree': 0.7,
 'learning_rate': 0.01,
 'max_depth': 5,
 'min_child_weight': 1,
 'n_estimators': 500,
 'objective': 'reg:squarederror',
 'subsample': 0.7}

**Best params:**

<!-- {'colsample_bytree': 0.7,
 'learning_rate': 0.1,
 'max_depth': 5,
 'min_child_weight': 3,
 'n_estimators': 100,
 'objective': 'reg:squarederror',
 'subsample': 0.5} -->
 
 {'colsample_bytree': 0.5,
 'learning_rate': 0.1,
 'max_depth': 3,
 'min_child_weight': 1,
 'n_estimators': 100,
 'objective': 'reg:squarederror',
 'subsample': 0.5}

In [31]:
model = XGBRegressor(
        colsample_bytree = 0.7,
        learning_rate = 0.01,
        max_depth = 5,
        min_child_weight = 1,
        n_estimators = 500,
        objective = 'reg:squarederror',
        subsample = 0.7)

%time model.fit(X_train, y_train, early_stopping_rounds=5, eval_set=[(X_test, y_test)], verbose=False)

y_pred = model.predict(X_test)

print("MAE: ", mean_absolute_error(y_test, y_pred))
print(f'R2: {r2_score(y_test, y_pred)}')

CPU times: user 2.8 s, sys: 173 ms, total: 2.97 s
Wall time: 1.16 s
MAE:  18862.67962328767
R2: 0.8970044139302908


In [27]:
def save_submission_xgboost(model_name, model, features):
    # read test data file using pandas
    test_data = pd.read_csv("./data/hpc_test.csv")

    # create test_X which comes from test_data but includes only the columns you used for prediction.
    # The list of columns is stored in a variable called features

    test_X = test_data[features]
    
    # make predictions which we will submit. 
    test_preds = model.predict(test_X)

    # The lines below shows you how to save your data in the format needed to score it in the competition
    output = pd.DataFrame({'Id': test_data.Id,
                           'SalePrice': test_preds})
    
    output_file = f'submission_{model_name}.csv'
    output.to_csv(output_file, index=False)
    
    print(f'{output_file} saved!')

In [32]:
save_submission_xgboost('xgboost', model, features)

submission_xgboost.csv saved!
